In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import math
import os
import sys
import logging

import numpy as np
import pandas as pd
import geopandas as gpd
import torch
import torch.nn as nn
import networkx as nx

from torch.utils.data import DataLoader, Dataset
from gensim.models import KeyedVectors
from node2vec import Node2Vec
from shapely.geometry import Point, MultiPolygon
from shapely.wkt import loads
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

**Load Dataset Features**

In [ ]:
try:
  from google.colab import drive
  logger.info("Running on Google Colab, reading dataset from drive")
  drive.mount("/content/drive")
  DATASET_PATH = "/content/drive/MyDrive/ECE2500/EdmontonFireRescueServicesData"
except:
  logger.info("Running locally, reading dataset from local file system")
  DATASET_PATH = "../dataset/EdmontonFireRescueServicesData"
  if not os.path.exists(DATASET_PATH):
    logger.critical(f"Cannot find dataset directory, place dataset in {DATASET_PATH}")
    exit(1)

UNIT_TRIP_PATH = os.path.join(DATASET_PATH, "EFRS_Unit_Trip_Summary.csv")
WEEKLY_EVENTS_PATH = os.path.join(DATASET_PATH, "weekly_events.csv")
NEIGHBOURHOOD_PATH = os.path.join(DATASET_PATH, "City_of_Edmonton_-_Neighbourhoods_20241022.csv")
NEIGHBOURHOOD_FEATURES_PATH = os.path.join(DATASET_PATH, "neighbourhood_features.csv")

logger.debug(f"Unit Trip: {UNIT_TRIP_PATH}")
logger.debug(f"Weekly Events: {WEEKLY_EVENTS_PATH}")
logger.debug(f"Neighbourhood: {NEIGHBOURHOOD_PATH}")
logger.debug(f"Neighbourhood Features: {NEIGHBOURHOOD_FEATURES_PATH}")

unit_trip_df = pd.read_csv(UNIT_TRIP_PATH)
weekly_events_df_all = pd.read_csv(WEEKLY_EVENTS_PATH)
neighbourhood_df = pd.read_csv(NEIGHBOURHOOD_PATH)
neighbourhood_feature_df = pd.read_csv(NEIGHBOURHOOD_FEATURES_PATH)

In [ ]:
event_list = ['Emergency Life Threatening - Immediate', 'Emergency Life Threatening', 'Emergency Non-Life Threatening', 'Potential Life Threatening', 'Non-Emergency']
weekly_events_df = weekly_events_df_all[weekly_events_df_all['Rc_description'].isin([i for i in event_list])]


**Data and Embedding**

In [ ]:
def generate_node2vec_embeddings(neighbourhood_info_df, node2vec_dim=32):
  """
  Generates Node2Vec embeddings for neighborhoods based on a sample adjacency graph.
  node2vec_dim: Dimension of the embeddings to be generated.
  """

  num_neighborhood_info = len(neighbourhood_info_df)
  G = nx.Graph()

  # Convert "Geometry Multipolygon" column to GeoSeries
  neighbourhood_info_df['geometry'] = gpd.GeoSeries.from_wkt(neighbourhood_info_df['Geometry Multipolygon'])
  neighbourhood_info_df['nid'] = range(num_neighborhood_info)

  # Assuming you have a way to define neighborhood connections based on proximity
  # You can use the geometry information for this.
  # Here's a placeholder for how you might connect neighborhoods based on proximity:

  for i in range(num_neighborhood_info):
    for j in range(i + 1, num_neighborhood_info):
      # Use the new 'geometry' column for spatial operations
      if neighbourhood_info_df['geometry'].iloc[i].intersects(neighbourhood_info_df['geometry'].iloc[j]):
        G.add_edge(i, j)

  # Alternatively, you could build a graph based on other criteria like sharing a boundary
  node2vec = Node2Vec(G, dimensions=node2vec_dim, walk_length=10, num_walks=100, p=1, q=1)
  node2vec_model = node2vec.fit()
  node2vec_embeddings_np = np.array([node2vec_model.wv[str(i)] for i in range(num_neighborhood_info)])
  node2vec_embeddings = torch.from_numpy(node2vec_embeddings_np)
  node2vec_emb_layer = nn.Embedding.from_pretrained(node2vec_embeddings, freeze=True)

  return node2vec_emb_layer


class Time2Vec(nn.Module):
  """
  Time2Vec embedding module for temporal features

  This captures both linear and periodic components for time-based features.
  """
  def __init__(self, input_dim, embed_dim, act_function=torch.sin):
    super(Time2Vec, self).__init__()
    self.embed_dim = embed_dim // input_dim  # Embedding dimension per time feature
    self.act_function = act_function       # Activation function for periodicity
    self.weight = nn.Parameter(torch.randn(input_dim, self.embed_dim))
    self.bias = nn.Parameter(torch.randn(input_dim, self.embed_dim))

  def forward(self, x):
    # Diagonal embedding for each time feature (day of week, hour, etc.)
    x = torch.diag_embed(x)
    x_affine = torch.matmul(x, self.weight) + self.bias
    x_affine_0, x_affine_remain = torch.split(x_affine, [1, self.embed_dim - 1], dim=-1)
    x_affine_remain = self.act_function(x_affine_remain)
    return torch.cat([x_affine_0, x_affine_remain], dim=-1).view(x.size(0), x.size(1), -1)


class NeighborhoodDataset(Dataset):
    def __init__(self, neighborhood_ids, time_features, building_type_ids, building_counts,
                 population, event_type_ids, equipment_ids, targets):
        self.neighborhood_ids = neighborhood_ids  # Tensor of input neighborhood_ids
        self.time_features = time_features  # Tensor of input time_features
        self.building_type_ids = building_type_ids  # Tensor of input building_type_ids
        self.building_counts = building_counts  # Tensor of input building_counts
        self.population = population  # Tensor of input population
        self.event_type_ids = event_type_ids  # Tensor of input event_type_ids
        self.equipment_ids = equipment_ids  # Tensor of input equipment_ids
        self.targets = targets    # Tensor of target values

    def __len__(self):
        return len(self.neighborhood_ids)  # Number of neighborhoods

    def __getitem__(self, idx):
        return (self.neighborhood_ids[idx], self.time_features[idx], self.building_type_ids[idx], self.building_counts[idx],
                self.population[idx], self.event_type_ids[idx], self.equipment_ids[idx], self.targets[idx])


class CombinedEmbedding(nn.Module):
  """
  Combined Embedding Module

  Combines embeddings from Node2Vec, Time2Vec, building type/counts, population, event type, and equipment.
  Projects the combined embedding to a target dimension (e.g., 64) for compatibility with transformer layers.
  """
  def __init__(self, node2vec_emb_layer, time2vec_embed_dim, time_feature_dim,
          num_building_types, building_type_embed_dim, population_embed_dim,
          num_event_types, event_type_embed_dim, num_equipment_types, equipment_embed_dim,
          target_embed_dim=64):  # Add target_embed_dim for projection
    super(CombinedEmbedding, self).__init__()

    # Embedding initialization code
    self.node2vec_emb_layer = node2vec_emb_layer  # Precomputed Node2Vec embeddings
    self.time2vec = Time2Vec(input_dim=time_feature_dim, embed_dim=time2vec_embed_dim)
    self.building_type_embedding = nn.Embedding(num_building_types, building_type_embed_dim)
    self.population_embedding = nn.Linear(1, population_embed_dim)
    # self.income_embedding = nn.Linear(1, income_embed_dim)
    self.event_type_embedding = nn.Embedding(num_event_types, event_type_embed_dim)
    self.equipment_embedding = nn.Embedding(num_equipment_types, equipment_embed_dim)

    # Compute the combined embedding dimension before projection
    # num_neighbourhood * 2 (month + year) * x
    self.projection_dim = (node2vec_emb_layer.embedding_dim + time2vec_embed_dim +
                    building_type_embed_dim + population_embed_dim +
                    event_type_embed_dim + equipment_embed_dim)

    # Projection layer to reduce to target_embed_dim
    self.projection_layer = nn.Linear(self.projection_dim, target_embed_dim)
    
  def forward(self, neighborhood_ids, time_features, building_type_ids, building_counts,
            population, event_type_ids, equipment_ids):
    # Generate embeddings
    spatial_embeddings = self.node2vec_emb_layer(neighborhood_ids).unsqueeze(1).repeat(1, time_features.size(1), 1)
    logger.debug(f"Spatial Embedding Shape: {spatial_embeddings.shape}")

    temporal_embeddings = self.time2vec(time_features)
    logger.debug(f"Temporal Embedding Shape: {temporal_embeddings.shape}")

    # Building embeddings
    building_type_embeds = self.building_type_embedding(building_type_ids)  # Shape: [batch_size, num_building_types, building_type_embed_dim]
    logger.debug(f"Building Type Embeds Shape: {building_type_embeds.shape}")

    # Adjust building_counts to match building_type_embed_dim
    building_counts = building_counts.unsqueeze(-1)  # Shape: [batch_size, 1, num_building_types, 1]
    building_counts = building_counts.repeat(1, 1, 1, self.building_type_embedding.embedding_dim)  # Match embed_dim
    logger.debug(f"Building Counts Shape after adjustment: {building_counts.shape}")

    # Multiply and aggregate
    building_embeddings = (building_type_embeds.unsqueeze(1) * building_counts).sum(dim=2)  # Shape: [batch_size, 1, building_type_embed_dim]
    building_embeddings = building_embeddings.repeat(1, time_features.size(1), 1)  # Match temporal dimension
    logger.debug(f"Building Embedding Shape: {building_embeddings.shape}")

    population_embeddings = self.population_embedding(population.unsqueeze(-1)).unsqueeze(1).repeat(1, time_features.size(1), 1)
    logger.debug(f"Population Embedding Shape: {population_embeddings.shape}")

    event_type_embeddings = self.event_type_embedding(event_type_ids)
    logger.debug(f"Event Type Embedding Shape: {event_type_embeddings.shape}")

    equipment_embeddings = self.equipment_embedding(equipment_ids)
    logger.debug(f"Equipment Embedding Shape: {equipment_embeddings.shape}")

    # Concatenate all embeddings
    combined_embedding = torch.cat([spatial_embeddings, temporal_embeddings, building_embeddings,
                                     population_embeddings, event_type_embeddings, equipment_embeddings], dim=-1)
    logger.debug(f"Combined Embedding Shape before Projection: {combined_embedding.shape}")

    # Project to target dimension
    combined_embedding = self.projection_layer(combined_embedding)
    return combined_embedding

class PositionalEncoding(nn.Module):
  """
  Positional Encoding Module
  """
  def __init__(self, embed_dim, max_len=7):  # 7 days in a week
    super(PositionalEncoding, self).__init__()
    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_dim, 2) * - (math.log(10000.0) / embed_dim))
    pe = torch.zeros(max_len, embed_dim)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    self.pe = pe.unsqueeze(0)  # Shape: (1, max_len, embed_dim)

  def forward(self, x):
    x = x + self.pe[:, :x.size(1), :].to(x.device)
    return x

**Transformer Model**

In [ ]:
# Transformer-based Emergency Event Predictor
class EmergencyEventPredictor(nn.Module):
  def __init__(self, embedding_module, embed_dim, num_heads, num_layers, max_len=7):
    super(EmergencyEventPredictor, self).__init__()

    # Embedding module (CombinedEmbedding) and positional encoding
    self.embedding_module = embedding_module
    self.positional_encoding = PositionalEncoding(embed_dim, max_len)

    # Transformer Encoder
    encoder_layer = nn.TransformerEncoderLayer(
        d_model=embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.1
    )
    self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    # Prediction head
    self.fc_out = nn.Linear(embed_dim, 1)  # Output: predicting the number of events

  def forward(self, neighborhood_ids, time_features, building_type_ids, building_counts,
              population, event_type_ids, equipment_ids):

    # Generate combined embeddings from the embedding module
    x = self.embedding_module(neighborhood_ids, time_features, building_type_ids, building_counts,
                              population, event_type_ids, equipment_ids)

    # Apply positional encoding
    x = self.positional_encoding(x)

    # Pass through transformer encoder
    x = self.transformer_encoder(x)

    # Prediction layer (we apply it to each element in the sequence)
    # Albert: predictions = self.fc_out(x).squeeze(-1)  # Shape: [batch_size]
    predictions = self.fc_out(x.squeeze(1)).squeeze(-1)  # Shape: [batch_size]

    return predictions

**Feature Extraction**

In [ ]:
build_type_list = [
    'Apartment_Condo_1_to_4_stories', 'Apartment_Condo_5_or_more_stories',
    'Duplex_Fourplex', 'Hotel_Motel',
    'Institution_Collective_Residence', 'Manufactured_Mobile_Home',
    'RV_Tent_Other', 'Row_House',
    'Single_Detached_House'
    ]
income_list = ['Low_Income', 'Low_medium_Income', 'Medium_Income', 'High_Income']
event_type_list = weekly_events_df['Rc_description'].unique()
unit_type_list = unit_trip_df['unityp'].unique()

num_neighborhoods = len(neighbourhood_feature_df)
num_income = len(income_list)
num_building_types = len(build_type_list)
num_event_types = len(event_type_list)
num_equipment_types = len(unit_type_list)

neighbourhood_mappings = weekly_events_df["Neighbourhood Number"].unique()
building_counts_np = neighbourhood_feature_df[build_type_list].fillna(0).astype(int).to_numpy()
population_np = neighbourhood_feature_df['Population'].fillna(0).astype(int).to_numpy()
income_np = neighbourhood_feature_df[income_list].fillna(0).astype(int).to_numpy()

**Transformer Model Initalization**

In [ ]:
# Embedding Parameter and Module

node2vec_dim = 32
time2vec_embed_dim = 64
time_feature_dim = 2  # week, year
building_type_embed_dim = 16
population_embed_dim = 8
event_type_embed_dim = 16
equipment_embed_dim = 16
target_embed_dim = 64

node2vec_emb_layer = generate_node2vec_embeddings(
    neighbourhood_info_df=neighbourhood_df,
    node2vec_dim=node2vec_dim
)

embedding_module = CombinedEmbedding(
    node2vec_emb_layer=node2vec_emb_layer,
    time2vec_embed_dim=time2vec_embed_dim,
    time_feature_dim=time_feature_dim,
    num_building_types=num_building_types,
    building_type_embed_dim=building_type_embed_dim,
    population_embed_dim=population_embed_dim,
    num_event_types=num_event_types,
    event_type_embed_dim=event_type_embed_dim,
    num_equipment_types=num_equipment_types,
    equipment_embed_dim=equipment_embed_dim,
    target_embed_dim=target_embed_dim
)

In [ ]:
# Define batch and mini-batch

mini_batch_size = 29
batch_size = 13

spatial_dimension = num_neighborhoods

# verify spatial_dimension = batch_size * mini_batch_size
assert spatial_dimension == batch_size * mini_batch_size

In [ ]:
# Neighborhood Features
neighborhood_ids = torch.arange(num_neighborhoods)
logger.info(f"neighborhood_ids shape {neighborhood_ids.shape}")

# Time Features
time_features = torch.zeros(spatial_dimension, 1, time_feature_dim) # [# years, # weeks]
for nid in range(num_neighborhoods):
    # Filter rows for the current neighborhood
    neighborhood_data = weekly_events_df[weekly_events_df["Neighbourhood Number"] == neighbourhood_mappings[nid]]

    if not neighborhood_data.empty:
        # Use "year" and "week_of_year" as features
        year_mean = neighborhood_data["year"].mean()
        week_mean = neighborhood_data["week_of_year"].mean()

        # Combine them into time features (e.g., [mean year, mean week])
        time_features[nid, 0, :] = torch.tensor([year_mean, week_mean])
    else:
        # Default to zero if no data for the neighborhood
        time_features[nid, 0, :] = torch.zeros(time_feature_dim)

logger.info(f"time_features shape: {time_features.shape}")

# Building Features
building_type_ids = torch.arange(num_building_types).repeat(spatial_dimension, 1)
logger.info(f"building_type_ids shape {building_type_ids.shape}")

# Building Counts
building_counts_np = neighbourhood_feature_df[build_type_list].fillna(0).to_numpy(dtype=np.int32)
building_counts = torch.from_numpy(building_counts_np).unsqueeze(1)  # Temporal dimension = 1
logger.info(f"building_counts shape {building_counts.shape}")

# Demographic Features
population = torch.from_numpy(population_np).float()
logger.info(f"population shape {population.shape}")

income = torch.from_numpy(income_np).float()
logger.info(f"income shape {income.shape}")

# Event Features
event_type_ids = torch.randint(0, num_event_types, (spatial_dimension, 1))  # Temporal dimension = 1
logger.info(f"event_type_ids shape {event_type_ids.shape}")

equipment_ids = torch.randint(0, num_equipment_types, (spatial_dimension, 1))  # Temporal dimension = 1
logger.info(f"equipment_ids shape {equipment_ids.shape}")

# Target Values
agg_data = weekly_events_df.groupby(['year', 'week_of_year', 'Neighbourhood Number']).agg({
    'Rc_description': ' '.join,  # Concatenate descriptions (or other aggregation if needed)
    'event_count': 'sum'         # Sum event counts
}).reset_index()

unique_week_year_combinations = agg_data[['year', 'week_of_year']].drop_duplicates()
targets = torch.zeros((num_neighborhoods,), dtype=torch.float32)

for i, neighbourhood in enumerate(neighbourhood_feature_df['Neighbourhood_Number']):
    neighbourhood_data = agg_data[agg_data['Neighbourhood Number'] == neighbourhood]
    # Sum of event counts for each (year, week_of_year) combination
    for k, (year, week) in enumerate(unique_week_year_combinations.itertuples(index=False)):
        event_count = neighbourhood_data[(neighbourhood_data['year'] == year) &
                                         (neighbourhood_data['week_of_year'] == week)]['event_count'].sum()
        targets[i] = event_count  # Aggregated for the entire week
logger.info(f"targets shape {targets.shape}")

**Transformer Model Training Loop**

In [ ]:
# Instantiate the DataLoader and EmergencyEventPredictor
dataset = NeighborhoodDataset(neighborhood_ids, time_features, building_type_ids, building_counts,
                              population, event_type_ids, equipment_ids, targets)
dataloader = DataLoader(dataset, batch_size=mini_batch_size, shuffle=True)

model = EmergencyEventPredictor(
    embedding_module=embedding_module,
    embed_dim=64,
    num_heads=4,
    num_layers=2
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.PoissonNLLLoss()

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
            _population, _event_type_ids, _equipment_ids, _targets) in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                             _building_counts, _population, _event_type_ids, _equipment_ids)

        # Compute loss
        loss = criterion(_predictions, _targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        mini_batch_loss = loss.item()
        epoch_loss += mini_batch_loss
        logger.debug(f"Epoch [{epoch+1}/{num_epochs}], Mini-batch [{i + 1}/{len(dataloader)}], Loss: {mini_batch_loss:.4f}")

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {(epoch_loss / len(dataloader)):.4f}")

# Save the trained model
torch.save(model.state_dict(), "transformer_model_weekly.pth")
logger.info("Model saved successfully.")


**Transformer Model Prediction**

In [ ]:
# Define model parameters
embed_dim = target_embed_dim  # Same as output dimension of CombinedEmbedding
num_heads = 4
num_layers = 2
max_len = 7  # Sequence length (e.g., 7 days for a weekly prediction)

# Instantiate the model
model = EmergencyEventPredictor(
    embedding_module=embedding_module,  # Replace with actual CombinedEmbedding instance
    embed_dim=embed_dim,
    num_heads=num_heads,
    num_layers=num_layers,
    max_len=max_len
)

# Forward pass to get predictions
predictions = model(
    neighborhood_ids=neighborhood_ids,
    time_features=time_features,
    building_type_ids=building_type_ids,
    building_counts=building_counts,
    population=population,
    event_type_ids=event_type_ids,
    equipment_ids=equipment_ids
)

logger.info(f"Predictions Shape: {predictions.shape}")
logger.info(f"Predictions: {predictions}")

**Model Diagram**

In [ ]:
# ! pip install torchviz
# from torchviz import make_dot
# make_dot(predictions, params=dict(model.named_parameters())).render("model_architecture", format="png")

**Output Visulization**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Detach predictions and convert to NumPy for visualization
predictions_np = predictions.detach().numpy()
predictions_np = predictions_np.reshape(-1, 1) # shape (377, 1)

# Define the plot
plt.figure(figsize=(12, 100))
sns.heatmap(predictions_np, annot=True, cmap="coolwarm", cbar=True, fmt=".2f",
            xticklabels=[f"Week {i+1}" for i in range(predictions_np.shape[1])],
            yticklabels=[f"Neighborhood {i+1}" for i in range(predictions_np.shape[0])])

# Add titles and labels
plt.title("Predicted Number of Events per Day for Each Neighborhood")
plt.xlabel("Day of the Week")
plt.ylabel("Neighborhood")
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Splitting neighborhoods into train and validation sets
train_indices, val_indices = train_test_split(
    np.arange(num_neighborhoods), test_size=0.2, random_state=42
)

# Prepare validation tensors
train_neighborhood_ids = neighborhood_ids[train_indices]
train_time_features = time_features[train_indices]
train_building_type_ids = building_type_ids[train_indices]
train_building_counts = building_counts[train_indices]
train_population = population[train_indices]
train_event_type_ids = event_type_ids[train_indices]
train_equipment_ids = equipment_ids[train_indices]
train_targets = targets[train_indices]

# Prepare validation tensors
val_neighborhood_ids = neighborhood_ids[val_indices]
val_time_features = time_features[val_indices]
val_building_type_ids = building_type_ids[val_indices]
val_building_counts = building_counts[val_indices]
val_population = population[val_indices]
val_event_type_ids = event_type_ids[val_indices]
val_equipment_ids = equipment_ids[val_indices]
val_targets = targets[val_indices]

# Create Validation Dataset and DataLoader
val_dataset = NeighborhoodDataset(
    neighborhood_ids=val_neighborhood_ids,
    time_features=val_time_features,
    building_type_ids=val_building_type_ids,
    building_counts=val_building_counts,
    population=val_population,
    event_type_ids=val_event_type_ids,
    equipment_ids=val_equipment_ids,
    targets=val_targets
)
train_dataset = NeighborhoodDataset(
    neighborhood_ids=train_neighborhood_ids,
    time_features=train_time_features,
    building_type_ids=train_building_type_ids,
    building_counts=train_building_counts,
    population=train_population,
    event_type_ids=train_event_type_ids,
    equipment_ids=train_equipment_ids,
    targets=train_targets
)

val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)

# Validation Loop
def validate_model(model, val_dataloader):
    model.eval()  # Set model to evaluation mode
    all_predictions = []
    all_targets = []

    with torch.no_grad():  # Disable gradient computation
        i = 0
        for (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
             _population, _event_type_ids, _equipment_ids, _targets) in val_dataloader:
            
            # Forward pass
            _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                                 _building_counts, _population, _event_type_ids, _equipment_ids)

            # Collect predictions and true values
            all_predictions.append(_predictions.cpu().numpy())
            all_targets.append(_targets.cpu().numpy())
    
    # Combine all batches
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Compute metrics
    print(f"all_targets {all_targets.shape}, all_predictions {all_predictions.shape}")
    mae = mean_absolute_error(all_targets, all_predictions)
    mse = mean_squared_error(all_targets, all_predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_targets, all_predictions)

    print(f"Validation Results:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

    return all_predictions, all_targets

# Scatter Plot: Predicted vs. True Values
def plot_predictions_vs_true(all_predictions, all_targets):
    plt.figure(figsize=(8, 8))
    plt.scatter(all_targets, all_predictions, alpha=0.5, edgecolor='k')
    plt.plot([all_targets.min(), all_targets.max()], [all_targets.min(), all_targets.max()], 'r--')
    plt.title("Predicted vs True Event Counts")
    plt.xlabel("True Event Counts")
    plt.ylabel("Predicted Event Counts")
    plt.grid()
    plt.show()

# Heatmap: Neighborhood Predictions Over Time
def plot_heatmap(all_predictions, neighborhood_ids):
    if len(all_predictions.shape) == 1:
        all_predictions = all_predictions.reshape(-1, 1) # shape (377, 1)

    plt.figure(figsize=(12, 20))
    sns.heatmap(all_predictions, annot=True, cmap="coolwarm", cbar=True, fmt=".2f",
                xticklabels=[f"Week {i+1}" for i in range(all_predictions.shape[1])],
                yticklabels=[f"Neighborhood {i}" for i in neighborhood_ids])
    plt.title("Predicted Event Counts per Neighborhood per Day")
    plt.xlabel("Days")
    plt.ylabel("Neighborhood")
    plt.show()

# Perform validation
all_predictions, all_targets = validate_model(model, val_dataloader)

# Visualize results
plot_predictions_vs_true(all_predictions, all_targets)
plot_heatmap(all_predictions, val_neighborhood_ids)


In [ ]:
x = np.arange(len(all_predictions))  # Positions for the first list
width = 0.4  # Width of the bars

# Create the plot
plt.figure(figsize=(30, 5))
plt.bar(x - width / 2, all_predictions, width=width, label='all_predictions', color='skyblue')
plt.bar(x + width / 2, all_targets, width=width, label='all_targets', color='orange')

# Add labels, title, and legend
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Side-by-Side Bar Plot of Two Lists')
plt.xticks(x, [f'Item {i}' for i in range(len(all_predictions))])  # Label x-axis ticks
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy() if isinstance(y_true, torch.Tensor) else y_true
    y_pred = y_pred.cpu().detach().numpy() if isinstance(y_pred, torch.Tensor) else y_pred
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2

# Lists to store metrics
train_losses, val_losses = [], []
train_maes, val_maes = [], []
train_rmses, val_rmses = [], []

# Training loop with validation
num_epochs = 10
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0
    train_preds, train_targets = [], []

    for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
            _population, _event_type_ids, _equipment_ids, _targets) in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                             _building_counts, _population, _event_type_ids, _equipment_ids)

        # Compute loss
        loss = criterion(_predictions, _targets)
        train_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

        # Collect predictions and targets for metrics
        train_preds.append(_predictions)
        train_targets.append(_targets)

    # Concatenate all training predictions and targets
    train_preds = torch.cat(train_preds)
    train_targets = torch.cat(train_targets)
    train_mae, train_mse, train_rmse, train_r2 = calculate_metrics(train_targets, train_preds)
    train_losses.append(train_loss / len(dataloader))
    train_maes.append(train_mae)
    train_rmses.append(train_rmse)

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {train_loss/len(dataloader):.4f}")
    logger.info(f"Training Metrics - MAE: {train_mae:.4f}, MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")

    # Validation phase
    model.eval()
    val_preds, val_targets = [], []
    val_loss = 0
    with torch.no_grad():
        for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
                _population, _event_type_ids, _equipment_ids, _targets) in enumerate(val_dataloader):
            
            # Forward pass
            _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                                 _building_counts, _population, _event_type_ids, _equipment_ids)

            # Compute loss
            loss = criterion(_predictions, _targets)
            val_loss += loss.item()

            # Collect predictions and targets for metrics
            val_preds.append(_predictions)
            val_targets.append(_targets)

    # Concatenate all validation predictions and targets
    val_preds = torch.cat(val_preds)
    val_targets = torch.cat(val_targets)
    val_mae, val_mse, val_rmse, val_r2 = calculate_metrics(val_targets, val_preds)
    val_losses.append(val_loss / len(val_dataloader))
    val_maes.append(val_mae)
    val_rmses.append(val_rmse)

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss/len(val_dataloader):.4f}")
    logger.info(f"Validation Metrics - MAE: {val_mae:.4f}, MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}, R²: {val_r2:.4f}")

# Save the trained model
torch.save(model.state_dict(), "transformer_model_weekly.pth")
logger.info("Model saved successfully.")

# Visualization
epochs = range(1, num_epochs + 1)

# Plot Losses
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses, label="Training Loss")
plt.plot(epochs, val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

# Plot MAE
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_maes, label="Training MAE")
plt.plot(epochs, val_maes, label="Validation MAE")
plt.xlabel("Epochs")
plt.ylabel("Mean Absolute Error")
plt.title("Training and Validation MAE")
plt.legend()
plt.show()

# Plot RMSE
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_rmses, label="Training RMSE")
plt.plot(epochs, val_rmses, label="Validation RMSE")
plt.xlabel("Epochs")
plt.ylabel("Root Mean Squared Error")
plt.title("Training and Validation RMSE")
plt.legend()
plt.show()

In [ ]:
# Define the calculate_accuracy function
def calculate_accuracy(y_true, y_pred, tolerance=0.5):
    """
    Calculate accuracy for regression by considering predictions
    within a certain tolerance as correct.

    Args:
        y_true: True target values (Tensor or NumPy array).
        y_pred: Predicted values from the model (Tensor or NumPy array).
        tolerance: Acceptable error margin.

    Returns:
        Accuracy as a percentage.
    """
    y_true = y_true.cpu().numpy() if isinstance(y_true, torch.Tensor) else y_true
    y_pred = y_pred.cpu().detach().numpy() if isinstance(y_pred, torch.Tensor) else y_pred

    correct = np.abs(y_pred - y_true) <= tolerance
    accuracy = np.mean(correct) * 100  # Convert to percentage
    return accuracy

# Training and validation loop
num_epochs = 100
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0
    train_preds, train_targets = [], []

    for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
            _population, _event_type_ids, _equipment_ids, _targets) in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                             _building_counts, _population, _event_type_ids, _equipment_ids)

        # Compute loss
        loss = criterion(_predictions, _targets)
        train_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

        # Collect predictions and targets for metrics
        train_preds.append(_predictions)
        train_targets.append(_targets)

    # Concatenate all training predictions and targets
    train_preds = torch.cat(train_preds)
    train_targets = torch.cat(train_targets)
    train_accuracy = calculate_accuracy(train_targets, train_preds)
    train_losses.append(train_loss / len(dataloader))
    train_accuracies.append(train_accuracy)

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

    # Validation phase
    model.eval()
    val_preds, val_targets = [], []
    val_loss = 0

    with torch.no_grad():
        for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
                _population, _event_type_ids, _equipment_ids, _targets) in enumerate(val_dataloader):
            
            # Forward pass
            _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                                 _building_counts, _population, _event_type_ids, _equipment_ids)

            # Compute loss
            loss = criterion(_predictions, _targets)
            val_loss += loss.item()

            # Collect predictions and targets for metrics
            val_preds.append(_predictions)
            val_targets.append(_targets)

    # Concatenate all validation predictions and targets
    val_preds = torch.cat(val_preds)
    val_targets = torch.cat(val_targets)
    val_accuracy = calculate_accuracy(val_targets, val_preds)
    val_losses.append(val_loss / len(val_dataloader))
    val_accuracies.append(val_accuracy)

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), "transformer_model_with_accuracy.pth")
logger.info("Model saved successfully.")

# Plot Loss
plt.figure(figsize=(12, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Training Loss")
plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
plt.show()

# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Training Accuracy")
plt.plot(range(1, num_epochs + 1), val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.grid()
plt.show()

# Variance-Bias Trade-off Analysis
bias = [train_loss - train_acc for train_loss, train_acc in zip(train_losses, train_accuracies)]
variance = [val_loss - val_acc for val_loss, val_acc in zip(val_losses, val_accuracies)]

# Plot Variance-Bias Trade-off
plt.figure(figsize=(12, 5))
plt.plot(range(1, num_epochs + 1), bias, label="Bias")
plt.plot(range(1, num_epochs + 1), variance, label="Variance")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.title("Variance-Bias Trade-off")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import logging


# Training and validation loop with plotting
def train_and_evaluate(model, train_loader, val_loader, optimizer, criterion, num_epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        train_loss = 0
        train_preds, train_targets = [], []

        for batch in train_loader:
            neighborhood_ids, time_features, building_type_ids, building_counts, population, event_type_ids, equipment_ids, targets = batch

            optimizer.zero_grad()

            # Forward pass
            predictions = model(neighborhood_ids, time_features, building_type_ids,
                                building_counts, population, event_type_ids, equipment_ids)

            # Compute loss
            loss = criterion(predictions, targets)
            train_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Store predictions and targets for accuracy calculation
            train_preds.append(predictions)
            train_targets.append(targets)

        # Calculate average training loss and accuracy
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        train_preds = torch.cat(train_preds)
        train_targets = torch.cat(train_targets)
        train_accuracy = calculate_accuracy(train_targets, train_preds)
        train_accuracies.append(train_accuracy)

        # Validation Phase
        model.eval()
        val_loss = 0
        val_preds, val_targets = [], []

        with torch.no_grad():
            for batch in val_loader:
                neighborhood_ids, time_features, building_type_ids, building_counts, population, event_type_ids, equipment_ids, targets = batch

                # Forward pass
                predictions = model(neighborhood_ids, time_features, building_type_ids,
                                    building_counts, population, event_type_ids, equipment_ids)

                # Compute loss
                loss = criterion(predictions, targets)
                val_loss += loss.item()

                # Store predictions and targets for accuracy calculation
                val_preds.append(predictions)
                val_targets.append(targets)

        # Calculate average validation loss and accuracy
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        val_preds = torch.cat(val_preds)
        val_targets = torch.cat(val_targets)
        val_accuracy = calculate_accuracy(val_targets, val_preds)
        val_accuracies.append(val_accuracy)

        # Logging
        logger.info(f"Epoch [{epoch+1}/{num_epochs}]")
        logger.info(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")
        logger.info(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%\n")

    # Plotting Training and Validation Loss
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plotting Training and Validation Accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, train_accuracies, label='Training Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Variance-Bias Trade-off Analysis
    # In regression, bias and variance can be analyzed by comparing training and validation errors.
    bias = np.array(train_losses)
    variance = np.array(val_losses) - np.array(train_losses)

    # Plotting Variance-Bias Trade-off
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, bias, label='Bias (Training Loss)')
    plt.plot(epochs, variance, label='Variance (Val Loss - Train Loss)')
    plt.title('Bias-Variance Trade-off')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)
    plt.show()


model = EmergencyEventPredictor(
    embedding_module=embedding_module,
    embed_dim=64,
    num_heads=4,
    num_layers=2
)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
num_epochs = 10
train_and_evaluate(model, train_loader, val_loader, optimizer, criterion, num_epochs)